In [1]:
from __future__ import division
import random
import pprint
import sys
import time
import numpy as np
from optparse import OptionParser
import pickle

from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Input
from keras.models import Model
from keras_frcnn import config, data_generators
from keras_frcnn import losses as losses
import keras_frcnn.roi_helpers as roi_helpers
from keras.utils import generic_utils


/home/superlee/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
sys.setrecursionlimit(40000)

parser = OptionParser()

parser.add_option("-p", "--path", dest="train_path", help="Path to training data.")
parser.add_option("-o", "--parser", dest="parser", help="Parser to use. One of simple or pascal_voc",
				default="pascal_voc")
parser.add_option("-n", "--num_rois", type="int", dest="num_rois", help="Number of RoIs to process at once.", default=32)
parser.add_option("--network", dest="network", help="Base network to use. Supports vgg or resnet50.", default='resnet50')
parser.add_option("--hf", dest="horizontal_flips", help="Augment with horizontal flips in training. (Default=false).", action="store_true", default=False)
parser.add_option("--vf", dest="vertical_flips", help="Augment with vertical flips in training. (Default=false).", action="store_true", default=False)
parser.add_option("--rot", "--rot_90", dest="rot_90", help="Augment with 90 degree rotations in training. (Default=false).",
				  action="store_true", default=False)
parser.add_option("--num_epochs", type="int", dest="num_epochs", help="Number of epochs.", default=2000)
parser.add_option("--config_filename", dest="config_filename", help=
				"Location to store all the metadata related to the training (to be used when testing).",
				default="config.pickle")
parser.add_option("--output_weight_path", dest="output_weight_path", help="Output path for weights.", default='./model_frcnn.hdf5')
parser.add_option("--input_weight_path", dest="input_weight_path", help="Input path for weights. If not specified, will try to load default weights provided by keras.")


<Option at 0x7f8c72e2f4a8: --input_weight_path>

In [3]:
from keras_frcnn.pascal_voc_parser import get_data
from keras_frcnn import vgg as nn

In [4]:
C = config.Config()

C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False

C.num_rois = 32
C.network = 'vgg'
C.base_net_weights = './input/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'


C.model_path = "./output/model_frcnn.hdf5"
train_path = "./input/dataset/VOCdevkit2007"

In [6]:
all_imgs, classes_count, class_mapping = get_data(train_path)  #function get_data muse be above, otherwise comes a FILE_NOT_FOUND error

Parsing annotation files
syntax error: line 1, column 0


In [7]:
if 'bg' not in classes_count:
	classes_count['bg'] = 0
	class_mapping['bg'] = len(class_mapping)

C.class_mapping = class_mapping

In [8]:
inv_map = {v: k for k, v in class_mapping.items()}

print('Training images per class:')
pprint.pprint(classes_count)
print('Num classes (including bg) = {}'.format(len(classes_count)))


Training images per class:
{'aeroplane': 331,
 'bg': 0,
 'bicycle': 418,
 'bird': 599,
 'boat': 398,
 'bottle': 634,
 'bus': 272,
 'car': 1644,
 'cat': 389,
 'chair': 1432,
 'cow': 356,
 'diningtable': 310,
 'dog': 538,
 'horse': 406,
 'motorbike': 390,
 'person': 5447,
 'pottedplant': 625,
 'sheep': 353,
 'sofa': 425,
 'train': 328,
 'tvmonitor': 367}
Num classes (including bg) = 21


In [9]:
config_output_filename = "./output/config.pickle"

with open(config_output_filename, 'wb') as config_f:
	pickle.dump(C,config_f)
	print('Config has been written to {}, and can be loaded when testing to ensure correct results'.format(config_output_filename))

Config has been written to ./output/config.pickle, and can be loaded when testing to ensure correct results


In [10]:
random.shuffle(all_imgs)

num_imgs = len(all_imgs)

train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

print('Num train samples {}'.format(len(train_imgs)))
print('Num val samples {}'.format(len(val_imgs)))

Num train samples 5011
Num val samples 0


In [12]:
val_imgs

[{'bboxes': [{'class': 'horse',
    'difficult': False,
    'x1': 87,
    'x2': 391,
    'y1': 38,
    'y2': 335},
   {'class': 'person',
    'difficult': False,
    'x1': 221,
    'x2': 317,
    'y1': 40,
    'y2': 180}],
  'filepath': './input/dataset/VOCdevkit2007/VOC2007/JPEGImages/004589.jpg',
  'height': 364,
  'imageset': 'test',
  'width': 480},
 {'bboxes': [{'class': 'aeroplane',
    'difficult': False,
    'x1': 5,
    'x2': 481,
    'y1': 95,
    'y2': 250},
   {'class': 'aeroplane',
    'difficult': False,
    'x1': 4,
    'x2': 166,
    'y1': 197,
    'y2': 245}],
  'filepath': './input/dataset/VOCdevkit2007/VOC2007/JPEGImages/007993.jpg',
  'height': 332,
  'imageset': 'test',
  'width': 500},
 {'bboxes': [{'class': 'dog',
    'difficult': False,
    'x1': 220,
    'x2': 496,
    'y1': 103,
    'y2': 333},
   {'class': 'dog',
    'difficult': False,
    'x1': 18,
    'x2': 491,
    'y1': 74,
    'y2': 333},
   {'class': 'person',
    'difficult': True,
    'x1': 1,
    'x

In [10]:
data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, nn.get_img_output_length, K.image_dim_ordering(), mode='train')
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,K.image_dim_ordering(), mode='val')

In [11]:
input_shape_img = (None, None, 3)

img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(None, 4))

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(shared_layers, roi_input, C.num_rois, nb_classes=len(classes_count), trainable=True)

model_rpn = Model(img_input, rpn[:2])
model_classifier = Model([img_input, roi_input], classifier)

# this is a model that holds both the RPN and the classifier, used to load/save weights for the models
model_all = Model([img_input, roi_input], rpn[:2] + classifier)


In [12]:
try:
	print('loading weights from {}'.format(C.base_net_weights))
	model_rpn.load_weights(C.base_net_weights,by_name=True)
	model_classifier.load_weights(C.base_net_weights, by_name=True)
except:
	print('Could not load pretrained model weights. Weights can be found in the keras application folder \
		https://github.com/fchollet/keras/tree/master/keras/applications')


loading weights from ./input/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5


In [13]:
optimizer = Adam(lr=1e-5)
optimizer_classifier = Adam(lr=1e-5)
model_rpn.compile(optimizer=optimizer, loss=[losses.rpn_loss_cls(num_anchors), losses.rpn_loss_regr(num_anchors)])
model_classifier.compile(optimizer=optimizer_classifier, loss=[losses.class_loss_cls, losses.class_loss_regr(len(classes_count)-1)], metrics={'dense_class_{}'.format(len(classes_count)): 'accuracy'})
model_all.compile(optimizer='sgd', loss='mae')

In [14]:
epoch_length = 1000
num_epochs = 2000
iter_num = 0

losses = np.zeros((epoch_length, 5))
rpn_accuracy_rpn_monitor = []
rpn_accuracy_for_epoch = []
#start_time = time.time()

best_loss = np.Inf

class_mapping_inv = {v: k for k, v in class_mapping.items()}


In [31]:
X, Y, img_data = next(data_gen_train)
loss_rpn = model_rpn.train_on_batch(X, Y)
P_rpn = model_rpn.predict_on_batch(X)

In [84]:
R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_dim_ordering(), use_regr=True, overlap_thresh=0.7, max_boxes=300)

In [90]:
X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)

In [116]:
neg_samples = np.where(Y1[0, :, -1] == 1)
pos_samples = np.where(Y1[0, :, -1] == 0)

In [117]:
pos_samples

(array([ 22,  24,  43,  89,  92, 117, 121, 151, 178]),)

In [125]:
X2

array([[[34, 10,  8,  8],
        [ 6, 11,  8,  8],
        [15, 20,  8,  8],
        [22,  7, 11, 23],
        [13, 13, 11, 23],
        [ 8, 11,  8,  8],
        [40, 10,  8,  8],
        [13, 23,  8,  8],
        [25, 11,  6, 11],
        [37, 10,  8,  8],
        [26,  6, 23, 11],
        [34, 12,  8,  8],
        [15, 23,  8,  8],
        [33, 11,  8,  8],
        [35, 11,  8,  8],
        [36, 12,  8,  8],
        [13, 21,  8,  8],
        [11, 16, 11, 20],
        [ 7, 12,  8,  8],
        [14, 22,  8,  8],
        [35, 13,  8,  8],
        [17, 17,  8,  8],
        [19, 15, 23, 11],
        [30, 10, 19, 11],
        [24, 14, 23, 11],
        [37, 15,  8,  8],
        [18,  4, 31, 32],
        [28, 12, 21, 11],
        [13,  2, 32, 32],
        [32, 11, 11,  6],
        [ 0,  0, 19, 31],
        [30,  7, 19, 11],
        [14, 24,  8,  8],
        [24, 16, 23, 11],
        [20,  7, 11, 23],
        [16, 22,  8,  8],
        [30,  2, 11, 23],
        [ 0,  9, 19, 11],
        [31,

In [15]:
print('Starting training')

vis = True

for epoch_num in range(num_epochs):

	#progbar = generic_utils.Progbar(epoch_length)
	print('Epoch {}/{}'.format(epoch_num + 1, num_epochs))

	while True:
		try:

			if len(rpn_accuracy_rpn_monitor) == epoch_length and C.verbose:
				mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
				rpn_accuracy_rpn_monitor = []
				print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(mean_overlapping_bboxes, epoch_length))
				if mean_overlapping_bboxes == 0:
					print('RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')

			X, Y, img_data = next(data_gen_train)

			loss_rpn = model_rpn.train_on_batch(X, Y)

			P_rpn = model_rpn.predict_on_batch(X)

			R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_dim_ordering(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
			# note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
			X2, Y1, Y2, IouS = roi_helpers.calc_iou(R, img_data, C, class_mapping)

			if X2 is None:
				rpn_accuracy_rpn_monitor.append(0)
				rpn_accuracy_for_epoch.append(0)
				continue

			neg_samples = np.where(Y1[0, :, -1] == 1)
			pos_samples = np.where(Y1[0, :, -1] == 0)

			if len(neg_samples) > 0:
				neg_samples = neg_samples[0]
			else:
				neg_samples = []

			if len(pos_samples) > 0:
				pos_samples = pos_samples[0]
			else:
				pos_samples = []
			
			rpn_accuracy_rpn_monitor.append(len(pos_samples))
			rpn_accuracy_for_epoch.append((len(pos_samples)))

			if C.num_rois > 1:
				if len(pos_samples) < C.num_rois//2:
					selected_pos_samples = pos_samples.tolist()
				else:
					selected_pos_samples = np.random.choice(pos_samples, C.num_rois//2, replace=False).tolist()
				try:
					selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=False).tolist()
				except:
					selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=True).tolist()

				sel_samples = selected_pos_samples + selected_neg_samples
			else:
				# in the extreme case where num_rois = 1, we pick a random pos or neg sample
				selected_pos_samples = pos_samples.tolist()
				selected_neg_samples = neg_samples.tolist()
				if np.random.randint(0, 2):
					sel_samples = random.choice(neg_samples)
				else:
					sel_samples = random.choice(pos_samples)

			loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

			losses[iter_num, 0] = loss_rpn[1]
			losses[iter_num, 1] = loss_rpn[2]

			losses[iter_num, 2] = loss_class[1]
			losses[iter_num, 3] = loss_class[2]
			losses[iter_num, 4] = loss_class[3]
            
			print("iter_num    = %d" % iter_num)
			print("loss_rpn1   = %f" % loss_rpn[1])
			print("loss_rpn2   = %f" % loss_rpn[2])
			print("loss_class1 = %f" % loss_class[1])
			print("loss_class2 = %f" % loss_class[2])
			print("loss_class3 = %f" % loss_class[3])

			iter_num += 1

			#progbar.update(iter_num, [('rpn_cls', np.mean(losses[:iter_num, 0])), ('rpn_regr', np.mean(losses[:iter_num, 1])),
									  #('detector_cls', np.mean(losses[:iter_num, 2])), ('detector_regr', np.mean(losses[:iter_num, 3]))])

			if iter_num == epoch_length:
				loss_rpn_cls = np.mean(losses[:, 0])
				loss_rpn_regr = np.mean(losses[:, 1])
				loss_class_cls = np.mean(losses[:, 2])
				loss_class_regr = np.mean(losses[:, 3])
				class_acc = np.mean(losses[:, 4])

				mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
				rpn_accuracy_for_epoch = []

				if C.verbose:
					print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(mean_overlapping_bboxes))
					print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
					print('Loss RPN classifier: {}'.format(loss_rpn_cls))
					print('Loss RPN regression: {}'.format(loss_rpn_regr))
					print('Loss Detector classifier: {}'.format(loss_class_cls))
					print('Loss Detector regression: {}'.format(loss_class_regr))
					print('Elapsed time: {}'.format(time.time() - start_time))

				curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
				iter_num = 0
				start_time = time.time()

				if curr_loss < best_loss:
					if C.verbose:
						print('Total loss decreased from {} to {}, saving weights'.format(best_loss,curr_loss))
					best_loss = curr_loss
					model_all.save_weights(C.model_path)

				break

		except Exception as e:
			print('Exception: {}'.format(e))
			continue

print('Training complete, exiting.')

Starting training
Epoch 1/2000
iter_num    = 0
loss_rpn1   = 7.063937
loss_rpn2   = 0.190339
loss_class1 = 3.044523
loss_class2 = 0.552696
loss_class3 = 0.000000
iter_num    = 1
loss_rpn1   = 5.691202
loss_rpn2   = 0.463145
loss_class1 = 2.955662
loss_class2 = 0.465231
loss_class3 = 0.750000
iter_num    = 2
loss_rpn1   = 5.827634
loss_rpn2   = 0.210163
loss_class1 = 2.796956
loss_class2 = 0.543838
loss_class3 = 0.875000
iter_num    = 3
loss_rpn1   = 4.655982
loss_rpn2   = 0.220306
loss_class1 = 2.792784
loss_class2 = 0.773137
loss_class3 = 0.500000
iter_num    = 4
loss_rpn1   = 6.717597
loss_rpn2   = 0.067981
loss_class1 = 2.452368
loss_class2 = 0.439869
loss_class3 = 0.750000
iter_num    = 5
loss_rpn1   = 6.313916
loss_rpn2   = 0.100959
loss_class1 = 2.055731
loss_class2 = 0.598408
loss_class3 = 0.750000
iter_num    = 6
loss_rpn1   = 8.675983
loss_rpn2   = 0.436199
loss_class1 = 1.947717
loss_class2 = 0.421698
loss_class3 = 0.781250
iter_num    = 7
loss_rpn1   = 12.322211
loss_rpn2   

KeyboardInterrupt: 